In [10]:
import pandas as pd

# Load the data into a pandas dataframe
df = pd.read_csv("credit_train.csv")  # Load the raw dataset from CSV
df = df.drop(columns=['Loan ID', 'Customer ID'])  # Drop ID columns (not useful for modeling)

# Define categorical columns to convert to 'category' data type
cat_cols = [
    'Loan Status',
    'Term',
    'Years in current job',
    'Home Ownership',
    'Purpose'
]
for col in cat_cols:
    df[col] = df[col].astype('category')  # Convert these columns to categorical type

# Define columns that should be treated as integers
int_cols = [
    'Credit Score',
    'Number of Open Accounts',
    'Number of Credit Problems',
    'Bankruptcies',
    'Tax Liens'
]
for col in int_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')  # Convert to integers; invalid values become NaN

# Define columns that should be treated as floating-point numbers
float_cols = [
    'Current Loan Amount',
    'Annual Income',
    'Monthly Debt',
    'Years of Credit History',
    'Months since last delinquent',
    'Current Credit Balance',
    'Maximum Open Credit'
]
for col in float_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to floats; invalid values become NaN

# Map 'Loan Status' to a binary numeric target: Fully Paid -> 0, Charged Off -> 1
df['Loan Status_n'] = df['Loan Status'].map({
    'Fully Paid': 0,
    'Charged Off': 1
})

df = df.drop(columns=['Loan Status'])  # Drop the original text-based target column

# Create a binary feature indicating whether a delinquency was ever recorded
df['Has_Delinquency'] = df['Months since last delinquent'].isna().astype(int)  # 1 = never delinquent

# Transform 'Months since last delinquent': add 1 to values, replace NaNs with 0
df['Months since last delinquent'] = df['Months since last delinquent'].apply(
    lambda x: 0 if pd.isna(x) else x + 1
)

# Fill missing numeric values with median (excluding 'Months since last delinquent')
for col in df.select_dtypes(include=['float64', 'int64', 'Int64']).columns:
    if col != 'Months since last delinquent':
        df[col] = df[col].fillna(df[col].median())

# Fill missing categorical values with the mode (most frequent value)
for col in df.select_dtypes(include='category').columns:
    df[col] = df[col].fillna(df[col].mode()[0])

# Output descriptive statistics for numeric columns
print(df.describe())

# Show the first few rows of the cleaned dataset
df.head()

# Download dataset
df.to_csv("credit_processed.csv", index=False)

       Current Loan Amount   Credit Score  Annual Income   Monthly Debt  \
count         1.005140e+05  100514.000000   1.005140e+05  100514.000000   
mean          1.170190e+07    1007.489514   1.338337e+06   18460.895674   
std           3.171309e+07    1330.570078   9.731830e+05   12144.885080   
min           1.080200e+04     585.000000   7.662700e+04       0.000000   
25%           1.799160e+05     711.000000   9.324250e+05   10245.845000   
50%           3.122460e+05     724.000000   1.174162e+06   16220.300000   
75%           5.232920e+05     738.000000   1.509968e+06   23959.475000   
max           1.000000e+08    7510.000000   1.655574e+08  435843.280000   

       Years of Credit History  Months since last delinquent  \
count            100514.000000                 100514.000000   
mean                 18.192498                     16.736972   
std                   6.997977                     23.374068   
min                   3.600000                      0.000000   
25% 